# Introduction 

This script is being used to do a complete takedown of the Lentimonas LCC4 strain characterized by Sichert et al 2020 (https://www.nature.com/articles/s41564-020-0720-2)

For this analysis we only focus on the transcriptome due to lack of transparency in the proteomic datasets. We use the **Lentimonas LCC4 Pacbio** genome for all analysis. We have uploaded all files associated with this genome to the box file.

# Step 1: Acquiring counts tables 

Due to the odd labelling method (Locus Tags) used by the authors, we proceeded to re-assess the transcriptomic reads uploaded to SRA (**#add number**). The overall process is 

1. Data download
2. Read Clean up 
3. Genome indexing & Salmon quasi mapping
4. Count tables 

The following code below is using bash terminal scripts. 

In [ ]:
###############STEP 1.1: Data download############################################################


# data download 
# lets get the names of SRA files associated with project number 
module run Kingfisher_v0.4.1-rev1 kingfisher annotate --bioprojects <insert> -o <insert> -f tsv
cut -f1 your_file.tsv > PRJ.txt


# downloading all SRA files 

#!/bin/bash
# Assuming your list is stored in a file named "input_list.txt"
input_list="input_list.txt"

# Loop through each item in the list
while IFS= read -r line; do
    # Run the module command with the specified arguments
    module run Kingfisher_v0.4.1-rev1 kingfisher get -r "$line" -m ena-ascp aws-http prefetch
done < "$input_list"

In [ ]:
###############STEP 1.2: Read Clean-Up ############################################################
# Check qualtiy 
fastqc *.fastq
mutliqc .

# Clean up the fasta files (for single end reads)
fastp -i <insert> -o <insert> --failed_out <insert>.txt --qualified_quality_phred 20 --unqualified_percent_limit 10 --thread 16

# Re-Check qualtiy 
fastqc *.fastq
mutliqc .


In [ ]:
###############STEP 1.3: Genome Indexing & Quasi mapping ############################################################
# Set the path to the directory containing the FASTQ files
fastq_dir=""

# Set the index path
index_path=""

# Set the output directory
output_dir=""

# Loop over each FASTQ file in the directory
for fastq_file in "$fastq_dir"/*.fastq; do
    # Extract the filename without extension
    filename=$(basename "$fastq_file" .fastq)
    
    # Run Salmon quantification for each FASTQ file
    salmon quant --index "$index_path" \
                 --libType A \
                 --unmatedReads "$fastq_file" \
                 -p 30 \
                 --output "$output_dir/${filename}_salmon_quant"
done

In [ ]:
###############STEP 1.4: Table Clean up ############################################################


#!/bin/bash

# Set the directory containing the directories
parent_directory="/path/to/parent_directory"

# Loop over each directory
for directory in "$parent_directory"/*/; do
    # Extract the directory name
    dir_name=$(basename "$directory")
    
    # Check if quant.sf file exists in the directory
    if [ -f "$directory/quant.sf" ]; then
        # Rename quant.sf to directory_name.tsv
        mv "$directory/quant.sf" "$directory/$dir_name.tsv"
        echo "Renamed quant.sf to $dir_name.tsv in $directory"
    else
        echo "quant.sf not found in $directory"
    fi
done

In [ ]:
# The main counts are gonna be in a file called quant.sf
# Rename the files and move to somewhere better 
#!/bin/bash

# Set the directory containing the directories
parent_directory="/path/to/parent_directory"

# Loop over each directory
for directory in "$parent_directory"/*/; do
    # Extract the directory name
    dir_name=$(basename "$directory")
    
    # Check if quant.sf file exists in the directory
    if [ -f "$directory/quant.sf" ]; then
        # Rename quant.sf to directory_name.tsv
        mv "$directory/quant.sf" "$directory/$dir_name.tsv"
        echo "Renamed quant.sf to $dir_name.tsv in $directory"
    else
        echo "quant.sf not found in $directory"
    fi
done

# Script to move 



#!/bin/bash

# Define the target directory to move .sf files
target_dir="new_directory"

# Create the target directory if it doesn't exist
mkdir -p "$target_dir"

# Iterate over directories starting with "SRR"
for dir in SRR*/; do
    # Check if directory contains .sf files
    if [ -n "$(find "$dir" -maxdepth 1 -type f -name '*.sf')" ]; then
        # Move all .sf files to the target directory
        mv "$dir"*.sf "$target_dir/"
        echo "Moved .sf files from $dir to $target_dir/"
    else
        echo "No .sf files found in $dir"
    fi
done

find /scratch/shrinivas/Sargassum/Salmon_output/LCC4/quant_files -type f -name '*.tsv' -exec sh -c 'filename=$(basename "{}" .tsv); awk -v filename="$filename" "BEGIN {FS=OFS=\"\t\"} NR==1 {gsub(/NumReads/, \"NumReads_\"filename); gsub(/TPM/, \"TPM_\"filename)} {print}" "{}" > "$filename"_fixed_headers.tsv' \;

At the end of this script we will obtain a counts table for each gene and its transcripts per million and the total number of reads. 

# Step 2: Annotating Reads
The next step is to annotated the complete Lentimonas genome. We use different tools to try and get a holistic assessment of what the reads are. 

1. GhostKoala
2. Diamond Blastp
3. Eggnogg-Mapper
4. InterProScan

This section uses online tools and linux bash tools


In [ ]:
###############STEP 2.1: GhostKoala ############################################################
'''
GhostKoala is an online tool link below 
https://www.kegg.jp/ghostkoala/
Settings used: genus_prokaryotes + viruses
'''

In [ ]:
###############STEP 2.2: Diamond Blastp ############################################################
/home/timothy/programs/DIAMOND_v2.1.8/bin/diamond blastp --ultra-sensitive --iterate --max-target-seqs 1 --evalue 0.00001 --query /scratch/shrinivas/Sargassum/Genomes/Lentimonas_Genomes/Lentimonas_LCC4_Pacbio/protein.faa --db /scratch/databases/ncbi/2022_07/nr.dmnd --out Sargassum_pacbio_blast.tsv --threads 60 --compress 1 --outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen stitle staxids sscinames sphylums skingdoms sskingdoms


In [ ]:
###############STEP 2.3: Eggnogg-Mapper ############################################################
/home/timothy/programs/eggnog-mapper-2.1.6/emapper.py -i <input> --cpu 48 --itype proteins -o <output> --data_dir /scratch/timothy/databases/eggnog-mapper-rel20211209

In [ ]:
###############STEP 2.4: InterProScanner ############################################################
#!/usr/bin/env bash

# Print all info to log file
exec 1> "${0}.log.$(date +%s)" 2>&1

#### Pre-run setup
source ~/scripts/script_setup.sh
set +eu; conda activate /home/timothy/miniconda3/envs/py38; set -eu

INTERPROSCAN="/home/timothy/programs/my_interproscan/interproscan-5.53-87.0/interproscan.sh"
NCPUS=24

#### Start Script
FAA="protein.faa"
run_cmd "${INTERPROSCAN} -dp --goterms --cpu ${NCPUS} --output-file-base ${FAA}.InterProScan --tempdir ${FAA}.InterProScan.temp --input ${FAA}"

# Step 3: Differential Gene Expression

For this section we used DESEQ2 on python to analyze the DGE of genes against the Mannose treatment. The authors of the paper us EdgeR, which is relatively lenient about replicates. The authors used LCC6 as a proxy replicate for LCC4 treatments.

However, for this re-analysis we combine the timepoints of each treatment as a replicate.

In [2]:
# packages and libraries used 
#libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

In [5]:
df = pd.read_csv('/Users/shrinivas/Desktop/Sargassum/PacBio_Transcriptome/Raw_counts_table.csv', index_col='Sample')
# note: use NumReads for DESEQ2 analysis
# Use TPM for DESEQ2 analysis

metadata = pd.read_csv(r'/Users/shrinivas/Desktop/Sargassum/Transcriptome_LCC4/Meta_data_WGCNA.csv', index_col='Sample')
transposed_data = df.T
genes_to_keep = transposed_data.columns[transposed_data.sum(axis=0) >= 10] # minimal reads cut off
transposed_data = transposed_data[genes_to_keep]
transposed_data = transposed_data.fillna(0).astype(int).astype(float)

In [ ]:
# Running DESEQ2
dds = DeseqDataSet(
    counts=transposed_data_sorted,
    metadata=metadata_sorted,
    design_factors=["Growth substrate"], # based on metadata
    refit_cooks=True)

dds.deseq2()
print(dds.varm["LFC"]) # check to see if code processed correctly


In [ ]:
# computing statistics. Log2FC, Wald t-test, and adjusted pvalues
stat_res = DeseqStats(dds)
stat_res_Y_vs_X = DeseqStats(dds, contrast=["Growth substrate", "iota", "Mannose"])
stat_res_Y_vs_X.summary()
res2 = stat_res_Y_vs_X.results_df
res2.to_csv('iota_vs_Mannose.csv')

# Step 4: Weighted Gene Co-Expression Analysis (On Going) 

This script is to build and assess modules of genes that are correlated with particular treatments. Further to build networks. 

While WGCNA has a python package, it is not as efficienty as the current R package. The code for the R package can be found at https://github.com/shrinivas-nandi/shrinivas-nandi/blob/main/R_files/WGCNA_Network.Rmd. 

**Note:** This is just a reference, only a couple of changes were made for this analysis, like file names etc



In [ ]:
```{r}
# packages used
library("genefilter")
library("RColorBrewer")
library("WGCNA")
library("flashClust")
library("gridExtra")
library("ComplexHeatmap")
library("goseq")
library("dplyr")
library("clusterProfiler")
library("pheatmap")
library("magrittr")
library("dendsort")
library("ggplot2")
library('vegan')
library('factoextra')
library('ggfortify')
library('naniar')
library('cowplot')
library("mixOmics")
library("tidyverse")
library("RVAideMemoire")
library("VennDiagram")
library("broom")
library("devtools")
library("tidyr")
```

# Step 5: Downstream Analysis
1. Selecting differentially expressed proteins (|FC| > 1.0 and adjpvalue < 0.05)
2. Plotting out data 

**Note**: Reference code for plots can be found on https://github.com/shrinivas-nandi/shrinivas-nandi/tree/main/Plots
